<br>
<br>

![](https://upload.wikimedia.org/wikipedia/en/5/5f/Western_Institute_of_Technology_and_Higher_Education_logo.png)

**InstitutoTecnológico y de Estudios Superiores de Occidente**

**Maestría Ciencia de Datos**

**Aprendizaje Profundo**

# Actividad 3 #

<br>
<br>

* * *

Estudiante: Daniel Nuño <br>
Profesor: Dr. Francisco Cervantes <br>
Fecha entrega: February 12, 2023 <br>

* * *

<br>
<br>

## Pre análisis

Creo que el uso de imágenes segmentas da una ligera ventaja ya que el fondo es negro, resalta la hoja, y la enfermedad. El primer paso es importar las imágenes:

- recesivamente importar las imágenes de la carpeta *segmented*.
    - primero intentar con escala de grises y después a color.
- cambiar el tamaño para que sea más manejable.
    - primero intentar con el tamaño original, después reduciendo el tamaño.
- rotar la matriz 90 grados 3 veces con la intención de generar 3 veces más datos.
- aplanarlos para tener una matriz de cuatro observaciones.
- usar el nombre de la carpeta para obtener las clasificaciones: tipo de hoja y estado de salud.

La matriz X esperada es:

- 56,304 x 4 = 225,216 filas
- dependiendo de la forma de importar de las imágenes X cantidad de columnas

La matriz Y esperada es:

- misma cantidad de filas que X
- 14 columnas por la especie más 26 columnas por enfermedades = 40 multiclase y multi clasificación.
    - 38 siendo la combinación de ambas. Tratar el modelo así tiene ventaja de ser solo multiclase.

Para la evolución de modelos será iterativo, modificando los hyperpárametros manualmente para entender cómo afectan el modelo. Pocos datos y pocas épocas para entrenar con el modelo cuando este decidido cual dirección pareciera ser el mejor.


## Crear conjunto de datos a partir de la estructura de carpetas

In [1]:
from PIL import Image
import numpy as np
import os

main_folder_path = 'C:/Users/nuno/Desktop/deep-learning-data/activity3/' + 'segmented/'

counter = 0
metadata_img = []
r_size = 28
arr = np.zeros((54306*4, r_size, r_size, 3)) #32*32*1 img size // 54306 * 4 rotations

for dirpath, dirnames, filenames in os.walk(main_folder_path):
    for filename in filenames:
        dirs_paths_state = [#os.path.join(dirpath, filename), #image full directory
                            str(os.path.basename(dirpath)).split('___')[0], #fruit
                            str(os.path.basename(dirpath)).split('___')[1] #state
                            ]
        img = Image.open(os.path.join(dirpath, filename)) #open image
        #img = img.convert('L') #convert to greyscale
        if img.getbands() == ('L',):
            img = img.convert('RGB') #convert to rgb to keep 3 channels if read as greyscale
        img = img.resize((r_size, r_size)) #resize

        for times in range(4):
            metadata_img.append(dirs_paths_state) #Y array
            arr[times + (counter*4)] = np.array(img)/255 #X array
            #arr[times + (counter*4)] = np.ravel(np.array(img)) #X array
            img = img.rotate(angle=90) #rotate image to generate
        
        counter += 1

img.close() #close connection

arr = arr.astype(np.float16)
metadata_img = np.array(metadata_img, dtype=str)

## Crear conjunto de entrenamiento, prueba y validación

In [2]:
from sklearn.preprocessing import OneHotEncoder

Y = np.char.add(metadata_img[:,0], metadata_img[:,1])
print((len(Y), len(set(Y))))

dummy_encoder_state = OneHotEncoder(sparse=False)
dummy_encoder_state.fit(Y.reshape(-1,1))
Y = dummy_encoder_state.transform(Y.reshape(-1,1))

#one could use:
    # tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # or parse_categorical_crossentropy as loss and sparse_categorical_accuracy as metric
#if do not one-hot encode your data

(217224, 38)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(arr, Y, test_size=0.9, random_state=27)
(X_train.shape, Y_train.shape)

((21722, 28, 28, 3), (21722, 38))

## Modelos red neuronal

Aunque el entrenamiento es el mismo para los diferentes modelos ajustando los hyperparametros, la red neuronal no es reproducible. Un buen modelo puede ser solo suerte.

In [10]:
from tensorflow import keras
from tensorflow.keras.layers import InputLayer, Conv2D, Flatten, Dense, Dropout

### Modelo 1

In [164]:
model1 = keras.Sequential([
    InputLayer(input_shape=(28, 28, 3)),
    Conv2D(1, kernel_size=(3,3), strides=1, padding='same', activation="relu"),
    Flatten(),
    Dense(38, activation="relu"),
    Dense(38, activation="softmax")
])

model1.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 28, 28, 1)         28        
                                                                 
 flatten_18 (Flatten)        (None, 784)               0         
                                                                 
 dense_49 (Dense)            (None, 38)                29830     
                                                                 
 dense_50 (Dense)            (None, 38)                1482      
                                                                 
Total params: 31,340
Trainable params: 31,340
Non-trainable params: 0
_________________________________________________________________


In [165]:
model1.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model1.fit(
        X_train,
        Y_train,
        epochs=2,
        )

Epoch 1/2
679/679 [==============================] - 30s 44ms/step - loss: 2.8927 - accuracy: 0.2225
Epoch 2/2
679/679 [==============================] - 32s 48ms/step - loss: 2.5552 - accuracy: 0.2771


### Modelo 2

Cambio drásticamente tamaño del filtro y kernel size.

In [166]:
model2 = keras.Sequential([
    InputLayer(input_shape=(28, 28, 3)),
    Conv2D(8, kernel_size=(6,6), strides=1, padding='same', activation="relu"),
    Flatten(),
    Dense(38, activation="relu"),
    Dense(38, activation="softmax")
])

model2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model2.fit(
        X_train,
        Y_train,
        epochs=2,
        )

Epoch 1/2
679/679 [==============================] - 54s 80ms/step - loss: 2.4745 - accuracy: 0.3291
Epoch 2/2
679/679 [==============================] - 55s 80ms/step - loss: 1.8187 - accuracy: 0.4768


In [167]:
model2.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 28, 28, 8)         872       
                                                                 
 flatten_19 (Flatten)        (None, 6272)              0         
                                                                 
 dense_51 (Dense)            (None, 38)                238374    
                                                                 
 dense_52 (Dense)            (None, 38)                1482      
                                                                 
Total params: 240,728
Trainable params: 240,728
Non-trainable params: 0
_________________________________________________________________


Se puede observar un cambio positivo en el accuracy. Aun se puede mejorar la primera capa oculta incrementando las neuronas hasta el tamaño de entrada previo (después de flatten).

### Modelo 3

Incrementa la primera capa oculta a 6272 neuronas.

In [168]:
model3 = keras.Sequential([
    InputLayer(input_shape=(28, 28, 3)),
    Conv2D(8, kernel_size=(6,6), strides=1, padding='same', activation="relu"),
    Flatten(),
    Dense(6272, activation="relu"),
    Dense(38, activation="softmax")
])

model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model3.fit(
        X_train,
        Y_train,
        epochs=2,
        )

Epoch 1/2
679/679 [==============================] - 169s 249ms/step - loss: 1.9206 - accuracy: 0.4501
Epoch 2/2
679/679 [==============================] - 169s 249ms/step - loss: 1.0915 - accuracy: 0.6642


El modelo mostro mejora pero fue mucho más lento.

### Modelo 4

Disminuye la cantidad de neuronas de la primera capa y aumenta la cantidad de capas ocultas.

In [182]:
model4 = keras.Sequential([
    InputLayer(input_shape=(28, 28, 3)),
    Conv2D(8, kernel_size=(6,6), strides=1, padding='same', activation="relu"),
    Flatten(),
    Dense(1500, activation="relu"),
    Dense(750, activation="relu"),
    Dense(375, activation="relu"),
    Dense(38, activation="softmax")
])

model4.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model4.fit(
        X_train,
        Y_train,
        epochs=2,
        )

Epoch 1/2
679/679 [==============================] - 66s 97ms/step - loss: 2.0437 - accuracy: 0.4123
Epoch 2/2
679/679 [==============================] - 66s 96ms/step - loss: 1.3098 - accuracy: 0.5996


El modelo se desempeñó ligeramente peor disminuyendo las neuronas de la primera capa y aumentando las capas ocultas. Fue sin embargo ligeramente más rápido.

### Modelo 5

Para el último modelo cambiar las funciones de activación en la segunda capa por *sigmoid*.

In [170]:
model5 = keras.Sequential([
    InputLayer(input_shape=(28, 28, 3)),
    Conv2D(8, kernel_size=(6,6), strides=1, padding='same', activation="relu"),
    Flatten(),
    Dense(1500, activation="sigmoid"),
    Dense(750, activation="relu"),
    Dense(375, activation="relu"),
    Dense(38, activation="softmax")
])

model5.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model5.fit(
        X_train,
        Y_train,
        epochs=2,
        )

Epoch 1/2
679/679 [==============================] - 67s 98ms/step - loss: 2.1945 - accuracy: 0.3718
Epoch 2/2
679/679 [==============================] - 68s 100ms/step - loss: 1.4598 - accuracy: 0.5523


El modelo tiene un desempeño peor que el modelo 3 y 4 por 10% y 5%. Aunque el modelo 4 tiene peor desempeño que el modelo 3 es más rápido.

### Modelo 6

Usando como base el modelo 4, cambiar optimizador por *nadam* y *sdg*. (Nadam is Adam, but with Nesterov momentum instead of ordinary momentum. The advantage of using Nesterov momentum instead of regular momentum is the same as it is in the SGD case.)

In [174]:
model6 = keras.Sequential([
    InputLayer(input_shape=(28, 28, 3)),
    Conv2D(8, kernel_size=(6,6), strides=1, padding='same', activation="relu"),
    Flatten(),
    Dense(1500, activation="relu"),
    Dense(750, activation="relu"),
    Dense(375, activation="relu"),
    Dense(38, activation="softmax")
])

model6.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model6.fit(
        X_train,
        Y_train,
        epochs=2,
        )

Epoch 1/2
679/679 [==============================] - 143s 209ms/step - loss: 2.0176 - accuracy: 0.4163
Epoch 2/2
679/679 [==============================] - 146s 215ms/step - loss: 1.2486 - accuracy: 0.6181


Nadam el último modelo trabaja bien, pero es casi igual de lento que el modelo 3 pero peor desempeño. SDG es más rápido, pero considerablemente peor.

#### AUC de los modelos

In [178]:
from keras.metrics import AUC

y_pred = model1.predict(X_train)
auc_validatio1 = AUC()
auc_validatio1.update_state(Y_train, y_pred)
auc_validatio1.result().numpy()

679/679 [==============================] - 2s 3ms/step


0.8881164

In [179]:
y_pred = model2.predict(X_train)
auc_validatio2 = AUC()
auc_validatio2.update_state(Y_train, y_pred)
auc_validatio2.result().numpy()

679/679 [==============================] - 25s 37ms/step


0.95536566

In [180]:
y_pred = model3.predict(X_train)
auc_validatio3 = AUC()
auc_validatio3.update_state(Y_train, y_pred)
auc_validatio3.result().numpy()

679/679 [==============================] - 6s 8ms/step


0.9831232

In [183]:
y_pred = model4.predict(X_train)
auc_validatio4 = AUC()
auc_validatio4.update_state(Y_train, y_pred)
auc_validatio4.result().numpy()

679/679 [==============================] - 6s 8ms/step


0.98561656

In [184]:
y_pred = model5.predict(X_train)
auc_validatio5 = AUC()
auc_validatio5.update_state(Y_train, y_pred)
auc_validatio5.result().numpy()

679/679 [==============================] - 6s 8ms/step


0.9774611

In [185]:
y_pred = model6.predict(X_train)
auc_validatio6 = AUC()
auc_validatio6.update_state(Y_train, y_pred)
auc_validatio6.result().numpy()

679/679 [==============================] - 6s 8ms/step


0.9700636

Los resultados indican que los modelos más confiables, respecto a la curva ROC, son el 3 y 4 por que son los valores más altos.

### Modelo 4 con entrenamiento completo.

Utilizar el modelo 4 con 80% de los datos y aumentando las epocas a 6.

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(arr, Y, test_size=0.18, random_state=27)
(X_train.shape, Y_train.shape)

((178123, 28, 28, 3), (178123, 38))

In [ ]:
model4 = keras.Sequential([
    InputLayer(input_shape=(28, 28, 3)),
    Conv2D(8, kernel_size=(6,6), strides=1, padding='same', activation="relu"),
    Flatten(),
    Dense(1500, activation="relu"),
    Dense(750, activation="relu"),
    Dropout(rate=0.01),
    Dense(375, activation="relu"),
    Dense(38, activation="softmax")
])

model4.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model4.fit(
        X_train,
        Y_train,
        epochs=2,
        )

In [14]:
test_loss, test_acc = model4.evaluate(X_test, Y_test, verbose=2)

1222/1222 - 10s - loss: 0.8840 - accuracy: 0.7953 - 10s/epoch - 8ms/step


El accuracy final es de 80%.

### Exportar modelo

In [16]:
model4.save('keras_model')

In [3]:
model = keras.models.load_model('keras_model')

### Crear pipeline

Este pipeline asume que las imágenes nuevas no se conoce la clasificación a priori.
- La imagen(es) vienen de una misma carpeta.
- Las imágenes son segmentadas.
- Se necesita Image de PIL, os, y array de numpy.
- No sabes cuantas imágenes hay.
- El tamaño de la imagen es convertida a 28x28x3.
- El modelo ya está construido y con pesos para usar *predict*.

In [14]:
test_folder_path = 'C:/Users/nuno/Desktop/deep-learning-data/activity3/' + 'test_pipeline/'

keys = ['Apple___Apple_scab', 'Apple___Black_rot',
       'Apple___Cedar_apple_rust', 'Apple___healthy',
       'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew',
       'Cherry_(including_sour)___healthy',
       'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
       'Corn_(maize)___Common_rust_',
       'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy',
       'Grape___Black_rot', 'Grape___Esca_(Black_Measles)',
       'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy',
       'Orange___Haunglongbing_(Citrus_greening)',
       'Peach___Bacterial_spot', 'Peach___healthy',
       'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy',
       'Potato___Early_blight', 'Potato___Late_blight',
       'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy',
       'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch',
       'Strawberry___healthy', 'Tomato___Bacterial_spot',
       'Tomato___Early_blight', 'Tomato___Late_blight',
       'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot',
       'Tomato___Spider_mites Two-spotted_spider_mite',
       'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
       'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']

In [71]:
def pipeline_new_images(main_folder_path, model_to_predict, keys):
    from PIL import Image
    import numpy as np
    import os

    results_array = []
    class_list = []
    files_names_list = []
    r_size = 28

    for dirpath, dirnames, filenames in os.walk(main_folder_path):
        for filename in filenames:
            files_names_list.append(os.path.join(dirpath, filename))
            
            img = Image.open(os.path.join(dirpath, filename)) #open image
            #convert to rgb to keep 3 channels if read as greyscale
            if img.getbands() == ('L',):
                img = img.convert('RGB')
            img = img.resize((r_size, r_size)) #resize
            arr = np.array(img)/255
            arr = np.expand_dims(arr, axis=0)
            result_ = model.predict(arr, verbose=0)

            results_array.append(result_)
            idx = np.argmax(result_)
            class_list.append(keys[idx])

    img.close() #close connection

    return results_array, class_list, files_names_list

In [72]:
r1, r2, r3 = pipeline_new_images(test_folder_path, model, keys)

### Conclusiones

En conclusión, el modelo parece sobre entrenado por que el accuracy en el entrenamiento es casi 15% mejor que la evaluación con la prueba.
El uso correcto de la convolución es más importante que la fuerza bruta. Más parámetros (neuronas y capas ocultas) ayudó a mejor los resultados.